# Environment Setup

In [1]:
from config import *

import gensim
import logging
import numpy as np

import nltk
from time import time
import re
import os
import math as m
import pandas as pd
from gensim import models

C:\Users\xt\Anaconda3\envs\py2\lib\site-packages\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

df_all=pd.read_csv(PROCESSINGTEXT_DIR+"/df_train_and_test_processed.csv", encoding="ISO-8859-1")
df_all1=pd.read_csv(PROCESSINGTEXT_DIR+"/df_product_descriptions_processed.csv", encoding="ISO-8859-1")
df_all2=pd.read_csv(PROCESSINGTEXT_DIR+"/df_attribute_bullets_processed.csv", encoding="ISO-8859-1")

df_all = pd.merge(df_all, df_all1, how="left", on="product_uid")
df_all = pd.merge(df_all, df_all2, how="left", on="product_uid")

In [3]:
for var in df_all.keys():
    df_all[var]=df_all[var].fillna("")

In [4]:
df_all

,id,product_title,product_uid,relevance,search_term,brand,product_title_simpleparsed,search_term_simpleparsed,search_term_parsed,is_query_misspelled,...,materials_in_product_description,product_description_stemmed,attribute_bullets,attribute_bullets_parsed,attribute_bullets_parsed_woBrand,brands_in_attribute_bullets,attribute_bullets_parsed_woBM,materials_in_attribute_bullets,attribute_bullets_tokens,attribute_bullets_stemmed
0,2,Simpson Strong-Tie 12-Gauge Angle,100001,3,angle bracket,Simpson Strong-Tie,simpson strong tie 12 gauge angle,angle bracket,angle bracket,1.000000,...,steel;various;zinc,angl make joint stronger also provid consist s...,Versatile connector for various 90ÃÂ° connect...,versatile connector for various 90 connections...,connector for various 90 connections and home ...,versatile,connector for 90 connections and home repair p...,steel;various,"[(connector, NN), (for, IN), (90, CD), (connec...",versatil connector various 90 connect home rep...
1,3,Simpson Strong-Tie 12-Gauge Angle,100001,2.5,l bracket,Simpson Strong-Tie,simpson strong tie 12 gauge angle,l bracket,l bracket,1.000000,...,steel;various;zinc,angl make joint stronger also provid consist s...,Versatile connector for various 90ÃÂ° connect...,versatile connector for various 90 connections...,connector for various 90 connections and home ...,versatile,connector for 90 connections and home repair p...,steel;various,"[(connector, NN), (for, IN), (90, CD), (connec...",versatil connector various 90 connect home rep...
2,9,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,100002,3,deck over,BEHR Premium Textured DeckOver,behr premium textured deckover 1 gal sc 141 tu...,deck over,deckover,0.941176,...,acrylic;concrete;composite;resin;wood,behr premium textur deckov innov solid color c...,"Revives wood and composite decks, railings, po...",revives wood and composite decks. railings. po...,revives wood and composite decks. railings. po...,behr,revives and decks. railings. porches and boat ...,acrylic;concrete;composite;wood,"[(revives, NNS), (and, CC), (decks, NNS), (., ...",reviv wood composit deck rail porch boat dock ...
3,16,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,100005,2.33,rain shower head,Delta,delta vero 1 handle shower only faucet trim ki...,rain shower head,rain showerhead,1.000000,...,,updat bath delta vero singl handl shower fauce...,"Includes the trim kit only, the rough-in kit (...",includes the trim kit only. the rough in kit. ...,includes the trim kit only. the rough in kit. ...,,includes the trim kit only. the rough in kit. ...,,"[(includes, VBZ), (the, DT), (trim, JJ), (kit,...",includ trim kit onli rough kit r10000 unbx sol...
4,17,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,100005,2.67,shower only faucet,Delta,delta vero 1 handle shower only faucet trim ki...,shower only faucet,shower only faucet,1.000000,...,,updat bath delta vero singl handl shower fauce...,"Includes the trim kit only, the rough-in kit (...",includes the trim kit only. the rough in kit. ...,includes the trim kit only. the rough in kit. ...,,includes the trim kit only. the rough in kit. ...,,"[(includes, VBZ), (the, DT), (trim, JJ), (kit,...",includ trim kit onli rough kit r10000 unbx sol...
5,18,Whirlpool 1.9 cu. ft. Over the Range Convectio...,100006,3,convection otr,Whirlpool,whirlpool 19 cu ft over the range convection m...,convection otr,convection otr,1.000000,...,,achiev delici result almost effortless whirlpo...,Spacious 1.9 cu. ft. capacity accommodates din...,spacious 1.9-cuft capacity accommodates dinner...,spacious 1.9-cuft capacity accommodates dinner...,,spacious 1.9-cuft capacity accommodates dinner...,,"[(spacious, JJ), (1.9-cuft, JJ), (capacity, NN...",spacious 1.9-cuft capac accommod dinner plate ...
6,20,Whirlpool 1.9 cu. ft. Over the Range Convectio...,100006,2.67,microwave over stove,Whirlpool,whirlpool 19 cu ft over the range convection m...,microwave over stove,microwave over stove,1.000000,...,,achiev delici result almost eff

# Construct word2vec models

In [8]:
search_term = df_all["search_term_stemmed"]
product_title = df_all["product_title_stemmed"]
product_desc = df_all["product_description_stemmed"]
brand = df_all["brand_parsed"]
material = df_all["material_parsed"]
bullet = df_all["attribute_bullets_stemmed"]

length = len(search_term )

vocab_lst_1 = list()
for i in range(length):
    p = search_term[i].split()+product_title[i].split()+product_desc[i].split()+brand[i].split()+material[i].split()+bullet[i].split()
    vocab_lst_1.append(p)

    
vocab_lst_2 = list()
for i in range(len(search_term)):
    p = search_term[i].split()
    vocab_lst_2 .append(p)

for i in range(len(product_title)):
    p = product_title[i].split()
    vocab_lst_2 .append(p)
    
for i in range(len(product_desc)):
    p = product_desc[i].split()
    vocab_lst_2 .append(p)
      
for i in range(len(bullet)):
    p = bullet[i].split()
    vocab_lst_2 .append(p)

In [11]:
model_1 = gensim.models.Word2Vec(vocab_lst_1, sg=1, window=10, sample=1e-5, negative=5, size=300)
model_2 = gensim.models.Word2Vec(vocab_lst_2, sg=1, window=10, sample=1e-5, negative=5, size=300)    

2017-03-09 15:13:29,671 : INFO : collecting all words and their counts
2017-03-09 15:13:29,674 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-03-09 15:13:30,049 : INFO : PROGRESS: at sentence #10000, processed 1807206 words, keeping 17409 word types
2017-03-09 15:13:30,334 : INFO : PROGRESS: at sentence #20000, processed 3398392 words, keeping 26632 word types
2017-03-09 15:13:30,720 : INFO : PROGRESS: at sentence #30000, processed 4939409 words, keeping 33864 word types
2017-03-09 15:13:31,114 : INFO : PROGRESS: at sentence #40000, processed 6420355 words, keeping 40624 word types
2017-03-09 15:13:31,448 : INFO : PROGRESS: at sentence #50000, processed 7839516 words, keeping 46719 word types
2017-03-09 15:13:31,799 : INFO : PROGRESS: at sentence #60000, processed 9353410 words, keeping 53093 word types
2017-03-09 15:13:32,170 : INFO : PROGRESS: at sentence #70000, processed 10944190 words, keeping 59071 word types
2017-03-09 15:13:32,601 : INFO : PROGR

In [12]:
product_title_similarity_1=list()

for i in range(length):
    search_term_word=search_term [i].split()
    product_title_word=product_title[i].split()
    d1=[]
    d2=[]
    for j in range(len(search_term_word)):
        if search_term_word[j] in model_1.wv.vocab:
            d1.append(search_term_word[j])
    for j in range(len(product_title_word)):
        if product_title_word[j] in model_1.wv.vocab:
            d2.append(product_title_word[j])
    if d1==[] or d2==[]:
        product_title_similarity_1.append(0)
    else:    
        product_title_similarity_1.append(model_1.n_similarity(d1,d2))

product_desc_similarity_1=list()
for i in range(length):
    search_term_word=search_term [i].split()
    product_desc_word=product_desc[i].split()
    d1=[]
    d2=[]
    for j in range(len(search_term_word)):
        if search_term_word[j] in model_1.wv.vocab:
            d1.append(search_term_word[j])
    for j in range(len(product_desc_word)):
        if product_desc_word[j] in model_1.wv.vocab:
            d2.append(product_desc_word[j])
    if d1==[] or d2==[]:
        product_desc_similarity_1.append(0)
    else:    
        product_desc_similarity_1.append(model_1.n_similarity(d1,d2))
        
bullet_similarity_1=list()
for i in range(length):
    search_term_word=search_term [i].split()
    bullet_word=bullet[i].split()
    d1=[]
    d2=[]
    for j in range(len(search_term_word)):
        if search_term_word[j] in model_1.wv.vocab:
            d1.append(search_term_word[j])
    for j in range(len(bullet_word)):
        if bullet_word[j] in model_1.wv.vocab:
            d2.append(bullet_word[j])
    if d1==[] or d2==[]:
        bullet_similarity_1.append(0)
    else:    
        bullet_similarity_1.append(model_1.n_similarity(d1,d2))
        
all_similarity_1=list()
for i in range(length):
    search_term_word=search_term [i].split()
    all_word=product_title[i].split()+product_desc[i].split()+brand[i].split()+material[i].split()+bullet[i].split()
    d1=[]
    d2=[]
    for j in range(len(search_term_word)):
        if search_term_word[j] in model_1.wv.vocab:
            d1.append(search_term_word[j])
    for j in range(len(all_word)):
        if all_word[j] in model_1.wv.vocab:
            d2.append(all_word[j])
    if d1==[] or d2==[]:
        all_similarity_1.append(0)
    else:    
        all_similarity_1.append(model_1.n_similarity(d1,d2))

In [13]:
product_title_similarity_2=list()

for i in range(length):
    search_term_word=search_term [i].split()
    product_title_word=product_title[i].split()
    d1=[]
    d2=[]
    for j in range(len(search_term_word)):
        if search_term_word[j] in model_2.wv.vocab:
            d1.append(search_term_word[j])
    for j in range(len(product_title_word)):
        if product_title_word[j] in model_2.wv.vocab:
            d2.append(product_title_word[j])
    if d1==[] or d2==[]:
        product_title_similarity_2.append(0)
    else:    
        product_title_similarity_2.append(model_2.n_similarity(d1,d2))

product_desc_similarity_2=list()
for i in range(length):
    search_term_word=search_term [i].split()
    product_desc_word=product_desc[i].split()
    d1=[]
    d2=[]
    for j in range(len(search_term_word)):
        if search_term_word[j] in model_2.wv.vocab:
            d1.append(search_term_word[j])
    for j in range(len(product_desc_word)):
        if product_desc_word[j] in model_2.wv.vocab:
            d2.append(product_desc_word[j])
    if d1==[] or d2==[]:
        product_desc_similarity_2.append(0)
    else:    
        product_desc_similarity_2.append(model_2.n_similarity(d1,d2))
        
bullet_similarity_2=list()
for i in range(length):
    search_term_word=search_term [i].split()
    bullet_word=bullet[i].split()
    d1=[]
    d2=[]
    for j in range(len(search_term_word)):
        if search_term_word[j] in model_2.wv.vocab:
            d1.append(search_term_word[j])
    for j in range(len(bullet_word)):
        if bullet_word[j] in model_2.wv.vocab:
            d2.append(bullet_word[j])
    if d1==[] or d2==[]:
        bullet_similarity_2.append(0)
    else:    
        bullet_similarity_2.append(model_2.n_similarity(d1,d2))
        
all_similarity_2=list()
for i in range(length):
    search_term_word=search_term [i].split()
    all_word=product_title[i].split()+product_desc[i].split()+brand[i].split()+material[i].split()+bullet[i].split()
    d1=[]
    d2=[]
    for j in range(len(search_term_word)):
        if search_term_word[j] in model_2.wv.vocab:
            d1.append(search_term_word[j])
    for j in range(len(all_word)):
        if all_word[j] in model_2.wv.vocab:
            d2.append(all_word[j])
    if d1==[] or d2==[]:
        all_similarity_2.append(0)
    else:    
        all_similarity_2.append(model_2.n_similarity(d1,d2))

In [14]:
remove_list=[str(x) for x in list(df_all.columns)]

for x in ['id']:
    remove_list.remove(x)

In [15]:
df_all["word2vec_product_title_1"] = product_title_similarity_1
df_all["word2vec_product_desc_1"] = product_desc_similarity_1
df_all["word2vec_bullet_1"] = bullet_similarity_1
df_all["word2vec_all_1"] = all_similarity_1

df_all["word2vec_product_title_2"] = product_title_similarity_2
df_all["word2vec_product_desc_2"] = product_desc_similarity_2
df_all["word2vec_bullet_2"] = bullet_similarity_2
df_all["word2vec_all_2"] = all_similarity_2

In [19]:
df_all

,id,word2vec_product_title_1,word2vec_product_desc_1,word2vec_bullet_1,word2vec_all_1,word2vec_product_title_2,word2vec_product_desc_2,word2vec_bullet_2,word2vec_all_2
0,2,0.458784,0.553634,0.533719,0.549535,0.483853,0.575611,0.551408,0.570924
1,3,0.311565,0.453877,0.465730,0.456067,0.268701,0.451644,0.472438,0.452242
2,9,0.763731,0.516316,0.439966,0.508309,0.757421,0.535903,0.478244,0.533710
3,16,0.561981,0.631503,0.629560,0.643102,0.576128,0.619519,0.609360,0.629267
4,17,0.821263,0.635256,0.535124,0.637509,0.813602,0.611602,0.515721,0.613078
5,18,0.816251,0.689544,0.681643,0.692760,0.838889,0.735763,0.730333,0.738616
6,20,0.723518,0.621791,0.629078,0.630419,0.689512,0.614489,0.622673,0.621921
7,21,0.777727,0.628754,0.642724,0.641073,0.764051,0.636438,0.649337,0.646895
8,23,0.772062,0.575134,0.529769,0.585648,0.776828,0.592917,0.556710,0.601942
9,27,0.612343,0.520279,0.574212,0.588948,0.531753,0.485249,0.542934,0.545603


In [20]:
df_all = df_all.drop(remove_list,axis=1)

ValueError: labels ['product_title' 'product_uid' 'relevance' 'search_term' 'brand'
 'product_title_simpleparsed' 'search_term_simpleparsed'
 'search_term_parsed' 'is_query_misspelled' 'product_title_parsed'
 'brand_parsed' 'material' 'material_parsed' 'search_term_parsed_woBrand'
 'brands_in_search_term' 'search_term_parsed_woBM'
 'materials_in_search_term' 'product_title_parsed_woBrand'
 'brands_in_product_title' 'product_title_parsed_woBM'
 'materials_in_product_title' 'search_term_tokens' 'product_title_tokens'
 'search_term_stemmed' 'search_term_stemmed_woBM'
 'search_term_stemmed_woBrand' 'product_title_stemmed'
 'product_title_stemmed_woBM' 'product_title_stemmed_woBrand'
 'product_description' 'product_description_parsed'
 'product_description_parsed_woBrand' 'brands_in_product_description'
 'product_description_parsed_woBM' 'materials_in_product_description'
 'product_description_stemmed' 'attribute_bullets'
 'attribute_bullets_parsed' 'attribute_bullets_parsed_woBrand'
 'bran

In [22]:
df_all.to_csv(FEATURES_DIR+"/df_word2vec.csv", index=False) 